In [1]:
import numpy as np 
import time
import os

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [2]:
from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d
import utils.layers as layers

In [3]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [4]:
print(X.shape, y.shape)

(83680, 160) (83680,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [6]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [8]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_validation, y_validation)
test = d.H_alphaSequences(X_test, y_test)

# CNN, bs=100, lr=1e-3, ReLU

In [9]:
CNN_1 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [10]:
CNN_1.eval()

Sequential(
  (0): Reshape()
  (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,), bias=False)
  (2): ReLU(inplace=True)
  (3): CBD1dBlock(
    (layer): Conv1d(16, 32, kernel_size=(4,), stride=(2,), bias=False)
    (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (4): CBD1dBlock(
    (layer): Conv1d(32, 64, kernel_size=(4,), stride=(2,), bias=False)
    (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (5): Flatten()
  (6): Linear(in_features=1152, out_features=256, bias=True)
  (7): ReLU(inplace=True)
  (8): Dropout(p=0.3, inplace=False)
  (9): Linear(in_features=256, out_features=4, bias=True)
)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

In [12]:
optimizer= torch.optim.Adam(CNN_1.parameters(), lr=1e-3)

In [13]:
m1 = Trainer(
        model=CNN_1,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_lr-1e-3_bs-100",
        verbose=True
        )

cuda:0


In [14]:
lh = m1(epochs=range(200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 200)], average_loss:0.5374, validation_loss:0.4251, val_accuracy:0.7865
Epoch [2/range(0, 200)], average_loss:0.4036, validation_loss:0.3762, val_accuracy:0.8764
Epoch [3/range(0, 200)], average_loss:0.3689, validation_loss:0.3678, val_accuracy:0.8427
Epoch [4/range(0, 200)], average_loss:0.3444, validation_loss:0.3304, val_accuracy:0.8652
Epoch [5/range(0, 200)], average_loss:0.3283, validation_loss:0.3454, val_accuracy:0.8876
Epoch [6/range(0, 200)], average_loss:0.3151, validation_loss:0.3198, val_accuracy:0.8652
Epoch [7/range(0, 200)], average_loss:0.3058, validation_loss:0.3105, val_accuracy:0.8652
Epoch [8/range(0, 200)], average_loss:0.3016, validation_loss:0.3023, val_accuracy:0.8539
Epoch [9/range(0, 200)], average_loss:0.2870, validation_loss:0.3217, val_accuracy:0.9101
Epoch [10/range(0, 200)], average_loss:0.2848, validation_loss:0.2907, val_accuracy:0.8427
Epoch [11/range(0, 200)], average_loss:0.2771, validation_loss:0.3105, val_accuracy:0.8989
Epoch [1

KeyboardInterrupt: 

## =>  early stopped

In [15]:
m1.tb.close()

In [284]:
m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float().to(m1.device)).cpu().detach(), axis=1)

In [285]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [286]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8824922913281428

In [287]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9275215105162524

In [288]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3848,  219,   30,  138],
       [ 323, 9981,   34,  107],
       [  16,   31,  411,   21],
       [ 190,   96,    8, 1283]], dtype=int64)

In [36]:
#torch.save(m1.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_lr-1e-3_bs-100_early_stopped.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon lo

## lower lr

In [38]:
CNN_2 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [39]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

optimizer= torch.optim.Adam(CNN_2.parameters(), lr=1e-4)

m2 = Trainer(
        model=CNN_2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-Conv1-BN-Drop-0.3_lr-1e-4_bs-100",
        verbose=True
        )

cuda:0


In [41]:
lh = m2(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 100)], average_loss:0.6440, validation_loss:0.5266, val_accuracy:0.7303
Epoch [2/range(0, 100)], average_loss:0.4912, validation_loss:0.4698, val_accuracy:0.7978
Epoch [3/range(0, 100)], average_loss:0.4423, validation_loss:0.4218, val_accuracy:0.8315
Epoch [4/range(0, 100)], average_loss:0.4107, validation_loss:0.4013, val_accuracy:0.8764
Epoch [5/range(0, 100)], average_loss:0.3823, validation_loss:0.3727, val_accuracy:0.8427
Epoch [6/range(0, 100)], average_loss:0.3643, validation_loss:0.3613, val_accuracy:0.8202
Epoch [7/range(0, 100)], average_loss:0.3501, validation_loss:0.3476, val_accuracy:0.9101
Epoch [8/range(0, 100)], average_loss:0.3382, validation_loss:0.3392, val_accuracy:0.8764
Epoch [9/range(0, 100)], average_loss:0.3304, validation_loss:0.3318, val_accuracy:0.8090
Epoch [10/range(0, 100)], average_loss:0.3259, validation_loss:0.3218, val_accuracy:0.8427
Epoch [11/range(0, 100)], average_loss:0.3165, validation_loss:0.3167, val_accuracy:0.9213
Epoch [1

Epoch [92/range(0, 100)], average_loss:0.2139, validation_loss:0.2585, val_accuracy:0.9326
Epoch [93/range(0, 100)], average_loss:0.2124, validation_loss:0.2680, val_accuracy:0.8764
Epoch [94/range(0, 100)], average_loss:0.2133, validation_loss:0.2644, val_accuracy:0.8764
Epoch [95/range(0, 100)], average_loss:0.2140, validation_loss:0.2703, val_accuracy:0.8315
Epoch [96/range(0, 100)], average_loss:0.2124, validation_loss:0.2744, val_accuracy:0.9438


KeyboardInterrupt: 

In [43]:
m2.tb.close()

In [289]:
m2.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float().to(m2.device)).cpu().detach(), axis=1)

In [290]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [291]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8550999939767723

In [292]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9128824091778203

In [293]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3792,  325,   18,  100],
       [ 364, 9978,   31,   72],
       [  26,   76,  365,   12],
       [ 246,  177,   11, 1143]], dtype=int64)

In [50]:
#torch.save(m2.model,"models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_lr-1e-4_bs-100_early_stopped.pt")

## bs = 32

In [51]:
CNN_3 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [53]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=32, shuffle=True)

optimizer= torch.optim.Adam(CNN_3.parameters(), lr=1e-3)

m3 = Trainer(
        model=CNN_3,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-Conv1-BN-Drop-0.3_lr-1e-3_bs-32",
        verbose=True
        )

cuda:0


In [54]:
lh = m3(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 100)], average_loss:0.5452, validation_loss:0.4540, val_accuracy:0.8462
Epoch [2/range(0, 100)], average_loss:0.4278, validation_loss:0.3929, val_accuracy:0.6923
Epoch [3/range(0, 100)], average_loss:0.3842, validation_loss:0.3652, val_accuracy:0.7692
Epoch [4/range(0, 100)], average_loss:0.3551, validation_loss:0.3412, val_accuracy:0.8462
Epoch [5/range(0, 100)], average_loss:0.3374, validation_loss:0.3424, val_accuracy:0.9231
Epoch [6/range(0, 100)], average_loss:0.3241, validation_loss:0.3233, val_accuracy:0.7692
Epoch [7/range(0, 100)], average_loss:0.3131, validation_loss:0.3736, val_accuracy:0.9231
Epoch [8/range(0, 100)], average_loss:0.3098, validation_loss:0.3128, val_accuracy:1.0000
Epoch [9/range(0, 100)], average_loss:0.2993, validation_loss:0.3142, val_accuracy:0.6154
Epoch [10/range(0, 100)], average_loss:0.2978, validation_loss:0.3103, val_accuracy:0.9231
Epoch [11/range(0, 100)], average_loss:0.2927, validation_loss:0.3003, val_accuracy:0.6923
Epoch [1

KeyboardInterrupt: 

In [55]:
m3.tb.close()

In [302]:
m3.model.train()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().to(m3.device)).cpu().detach(), axis=1)

In [303]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [304]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8634819654555002

In [305]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9144956978967496

In [306]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3765,  344,   32,   94],
       [ 373, 9961,   34,   77],
       [  22,   34,  394,   29],
       [ 251,  131,   10, 1185]], dtype=int64)

In [61]:
#torch.save(m3.model,"models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_lr-1e-3_bs-32_early_stopped.pt")

# Mark II

In [64]:
CNN_4 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.Flatten(out_features=32*38),
            nn.Linear(in_features=32*38, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [65]:
CNN_4.eval()

Sequential(
  (0): Reshape()
  (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,), bias=False)
  (2): ReLU(inplace=True)
  (3): CBD1dBlock(
    (layer): Conv1d(16, 32, kernel_size=(4,), stride=(2,), bias=False)
    (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (4): Flatten()
  (5): Linear(in_features=1216, out_features=256, bias=True)
  (6): ReLU(inplace=True)
  (7): Dropout(p=0.3, inplace=False)
  (8): Linear(in_features=256, out_features=4, bias=True)
)

In [66]:
optimizer= torch.optim.Adam(CNN_4.parameters(), lr=1e-3)

In [68]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=32, shuffle=True)

In [69]:
m4 = Trainer(
        model=CNN_4,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.II-Conv1-BN-Drop-0.3_lr-1e-3_bs-32",
        verbose=True
        )

cuda:0


In [70]:
lh = m4(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.5174, validation_loss:0.4507, val_accuracy:0.8462
Epoch [2/range(0, 75)], average_loss:0.4247, validation_loss:0.4369, val_accuracy:0.8462
Epoch [3/range(0, 75)], average_loss:0.3887, validation_loss:0.3783, val_accuracy:0.9231
Epoch [4/range(0, 75)], average_loss:0.3629, validation_loss:0.3936, val_accuracy:0.7692
Epoch [5/range(0, 75)], average_loss:0.3518, validation_loss:0.3554, val_accuracy:0.9231
Epoch [6/range(0, 75)], average_loss:0.3395, validation_loss:0.3745, val_accuracy:0.7692
Epoch [7/range(0, 75)], average_loss:0.3283, validation_loss:0.3444, val_accuracy:0.9231
Epoch [8/range(0, 75)], average_loss:0.3214, validation_loss:0.3267, val_accuracy:0.8462
Epoch [9/range(0, 75)], average_loss:0.3153, validation_loss:0.3502, val_accuracy:0.7692
Epoch [10/range(0, 75)], average_loss:0.3128, validation_loss:0.3277, val_accuracy:0.7692
Epoch [11/range(0, 75)], average_loss:0.3039, validation_loss:0.3458, val_accuracy:0.6923
Epoch [12/range(0, 

KeyboardInterrupt: 

# bigger dropout prob

In [71]:
CNN_5 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.Dropout(p=0.2),
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.5),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.5),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=256, out_features=4)
)

In [72]:
CNN_5.eval()

Sequential(
  (0): Reshape()
  (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,), bias=False)
  (2): Dropout(p=0.2, inplace=False)
  (3): ReLU(inplace=True)
  (4): CBD1dBlock(
    (layer): Conv1d(16, 32, kernel_size=(4,), stride=(2,), bias=False)
    (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (5): CBD1dBlock(
    (layer): Conv1d(32, 64, kernel_size=(4,), stride=(2,), bias=False)
    (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (6): Flatten()
  (7): Linear(in_features=1152, out_features=256, bias=True)
  (8): ReLU(inplace=True)
  (9): Dropout(p=0.5, inplace=False)
  (10): Linear(in_features=256, out_features=4, bias=True)
)

In [93]:
optimizer= torch.optim.Adam(CNN_5.parameters(), lr=1e-3)

In [94]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

In [95]:
m5 = Trainer(
        model=CNN_5,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-Conv1-BN-Drop-0.5_lr-1e-3_bs-100",
        verbose=True
        )

cuda:0


In [96]:
lh = m5(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.6236, validation_loss:0.5448, val_accuracy:0.7865
Epoch [2/range(0, 75)], average_loss:0.5172, validation_loss:0.5010, val_accuracy:0.8539
Epoch [3/range(0, 75)], average_loss:0.4793, validation_loss:0.4666, val_accuracy:0.8090
Epoch [4/range(0, 75)], average_loss:0.4537, validation_loss:0.4433, val_accuracy:0.8539
Epoch [5/range(0, 75)], average_loss:0.4406, validation_loss:0.4461, val_accuracy:0.8652
Epoch [6/range(0, 75)], average_loss:0.4266, validation_loss:0.4307, val_accuracy:0.8989
Epoch [7/range(0, 75)], average_loss:0.4128, validation_loss:0.4170, val_accuracy:0.8764
Epoch [8/range(0, 75)], average_loss:0.4080, validation_loss:0.4101, val_accuracy:0.8876
Epoch [9/range(0, 75)], average_loss:0.3981, validation_loss:0.3932, val_accuracy:0.8876
Epoch [10/range(0, 75)], average_loss:0.3894, validation_loss:0.3896, val_accuracy:0.8202
Epoch [11/range(0, 75)], average_loss:0.3866, validation_loss:0.4056, val_accuracy:0.8315
Epoch [12/range(0, 

In [98]:
m5.tb.close()

In [311]:
m5.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m5.model(torch.tensor(X_test).float().to(m5.device)).cpu().detach(), axis=1)

In [312]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [313]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8639913837998829

In [314]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9030831739961759

In [315]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3902,  207,   22,  104],
       [ 733, 9592,   31,   89],
       [  26,   35,  402,   16],
       [ 260,   88,   11, 1218]], dtype=int64)

# Mark IV

In [115]:
CNN_6 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0, bias=False), # n*1*160 -> n*16*156
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2), #n*16*156 -> n*16*78
    
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False),# n*16*78->n*32*74
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2),#n*32*74->n*32*37
    
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=0, bias=False), # n*32*37 -> n*64*17
            nn.ReLU(inplace=True),
            layers.Flatten(out_features=64*17),
            nn.Dropout(p=0.3),
    
            nn.Linear(in_features=64*17, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
    
            nn.Linear(in_features=256, out_features=4)
)

In [116]:
optimizer= torch.optim.Adam(CNN_6.parameters(), lr=1e-3)

In [117]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

In [118]:
m6 = Trainer(
        model=CNN_6,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-Conv1-MaxP-2_lr-1e-3_bs-100",
        verbose=True
        )

cuda:0


In [119]:
lh = m6(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.5998, validation_loss:0.4857, val_accuracy:0.7978
Epoch [2/range(0, 75)], average_loss:0.4409, validation_loss:0.4053, val_accuracy:0.8652
Epoch [3/range(0, 75)], average_loss:0.3910, validation_loss:0.3752, val_accuracy:0.8876
Epoch [4/range(0, 75)], average_loss:0.3522, validation_loss:0.3342, val_accuracy:0.8764
Epoch [5/range(0, 75)], average_loss:0.3243, validation_loss:0.3279, val_accuracy:0.9101
Epoch [6/range(0, 75)], average_loss:0.3082, validation_loss:0.3139, val_accuracy:0.8315
Epoch [7/range(0, 75)], average_loss:0.2967, validation_loss:0.2950, val_accuracy:0.8764
Epoch [8/range(0, 75)], average_loss:0.2872, validation_loss:0.2841, val_accuracy:0.8876
Epoch [9/range(0, 75)], average_loss:0.2767, validation_loss:0.2885, val_accuracy:0.9438
Epoch [10/range(0, 75)], average_loss:0.2741, validation_loss:0.2889, val_accuracy:0.9101
Epoch [11/range(0, 75)], average_loss:0.2655, validation_loss:0.2753, val_accuracy:0.8989
Epoch [12/range(0, 

# Mark V

In [125]:
CNN_7 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False), # n*1*160 -> n*32*156
            nn.ELU(inplace=True),
            nn.MaxPool1d(kernel_size=3), #n*32*156 -> n*32*52
    
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0, bias=False),# n*32*52->n*64*48
            nn.ELU(inplace=True),
            layers.Flatten(out_features=48*64),

            nn.Linear(in_features=48*64, out_features=126),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.3),
    
            nn.Linear(in_features=126, out_features=4)
)

In [126]:
optimizer= torch.optim.Adam(CNN_7.parameters(), lr=1e-4)

In [127]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=50, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=50, shuffle=True)

In [128]:
m7 = Trainer(
        model=CNN_7,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.V_Conv1-MaxP-3_lr-1e-4_bs-50",
        verbose=True
        )

cuda:0


In [129]:
lh = m7(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.7459, validation_loss:0.6490, val_accuracy:0.7949
Epoch [2/range(0, 75)], average_loss:0.6120, validation_loss:0.5867, val_accuracy:0.7692
Epoch [3/range(0, 75)], average_loss:0.5626, validation_loss:0.5539, val_accuracy:0.7949
Epoch [4/range(0, 75)], average_loss:0.5280, validation_loss:0.5318, val_accuracy:0.8462
Epoch [5/range(0, 75)], average_loss:0.5090, validation_loss:0.4936, val_accuracy:0.8205
Epoch [6/range(0, 75)], average_loss:0.4914, validation_loss:0.4978, val_accuracy:0.8974
Epoch [7/range(0, 75)], average_loss:0.4798, validation_loss:0.4967, val_accuracy:0.8205
Epoch [8/range(0, 75)], average_loss:0.4680, validation_loss:0.4703, val_accuracy:0.8974
Epoch [9/range(0, 75)], average_loss:0.4592, validation_loss:0.4584, val_accuracy:0.9487
Epoch [10/range(0, 75)], average_loss:0.4481, validation_loss:0.4514, val_accuracy:0.7949
Epoch [11/range(0, 75)], average_loss:0.4441, validation_loss:0.4345, val_accuracy:0.8974
Epoch [12/range(0, 

In [130]:
m7.tb.close()

In [131]:
with torch.no_grad():
    x_pred = np.argmax(m7.model(torch.tensor(X_test).float().to(m7.device)).cpu().detach(), axis=1)

In [132]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [133]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.830646179354908

In [134]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8822299235181644

In [135]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3385,  692,   36,  122],
       [ 423, 9803,   60,  159],
       [  22,   29,  410,   18],
       [ 227,  157,   26, 1167]], dtype=int64)

# LeNet

In [146]:
CNN_8 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0, bias=True), # n*1*160 -> n*6*156
            nn.Tanh(),
            nn.AvgPool1d(kernel_size=2), #n*32*156 -> n*32*78
    
            nn.Conv1d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True),# n*6*78->n*16*74
            nn.Tanh(),
            nn.AvgPool1d(kernel_size=2), #n*16*74 -> n*16*37
    
            layers.Flatten(out_features=16*37),

            nn.Linear(in_features=16*37, out_features=120),
            nn.Tanh(),
            
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),

    
            nn.Linear(in_features=84, out_features=4)
)

In [147]:
optimizer= torch.optim.Adam(CNN_8.parameters(), lr=1e-3)

In [143]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=50, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=50, shuffle=True)

In [144]:
m8 = Trainer(
        model=CNN_8,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-LeNet_lr-1e-3_bs-50",
        verbose=True
        )

cuda:0


In [152]:
lh = m8(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.5597, validation_loss:0.4871, val_accuracy:0.7436
Epoch [2/range(0, 75)], average_loss:0.4787, validation_loss:0.4659, val_accuracy:0.7436
Epoch [3/range(0, 75)], average_loss:0.4548, validation_loss:0.4620, val_accuracy:0.8205
Epoch [4/range(0, 75)], average_loss:0.4379, validation_loss:0.4613, val_accuracy:0.7692
Epoch [5/range(0, 75)], average_loss:0.4292, validation_loss:0.4319, val_accuracy:0.8205
Epoch [6/range(0, 75)], average_loss:0.4232, validation_loss:0.4205, val_accuracy:0.8974
Epoch [7/range(0, 75)], average_loss:0.4156, validation_loss:0.4193, val_accuracy:0.8462
Epoch [8/range(0, 75)], average_loss:0.4113, validation_loss:0.4171, val_accuracy:0.7436
Epoch [9/range(0, 75)], average_loss:0.4077, validation_loss:0.4116, val_accuracy:0.8462
Epoch [10/range(0, 75)], average_loss:0.4053, validation_loss:0.4222, val_accuracy:0.7949
Epoch [11/range(0, 75)], average_loss:0.4036, validation_loss:0.4099, val_accuracy:0.8718
Epoch [12/range(0, 

KeyboardInterrupt: 

In [153]:
m8.tb.close()

# Full Conv

In [154]:
CNN_9 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False), # n*1*160 -> n*32*156
            nn.ELU(inplace=True),
            nn.MaxPool1d(kernel_size=3), #n*32*156 -> n*32*52
    
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0, bias=False),# n*32*52->n*64*48
            nn.ELU(inplace=True),
    
            layers.Flatten(out_features=48*64),
            nn.Linear(in_features=48*64, out_features=4)
)

In [155]:
optimizer= torch.optim.Adam(CNN_9.parameters(), lr=1e-3)

In [156]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

In [157]:
m9 = Trainer(
        model=CNN_9,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-full_conv_lr-1e-3_bs-100",
        verbose=True
        )

cuda:0


In [172]:
lh = m9(epochs=range(100,125), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/range(100, 125)], average_loss:0.2792, validation_loss:0.3716, val_accuracy:0.8764
Epoch [102/range(100, 125)], average_loss:0.2825, validation_loss:0.3295, val_accuracy:0.9101
Epoch [103/range(100, 125)], average_loss:0.2789, validation_loss:0.3602, val_accuracy:0.8989
Epoch [104/range(100, 125)], average_loss:0.2767, validation_loss:0.3288, val_accuracy:0.8652
Epoch [105/range(100, 125)], average_loss:0.2756, validation_loss:0.3328, val_accuracy:0.8764
Epoch [106/range(100, 125)], average_loss:0.2737, validation_loss:0.3647, val_accuracy:0.8764
Epoch [107/range(100, 125)], average_loss:0.2756, validation_loss:0.3260, val_accuracy:0.8989
Epoch [108/range(100, 125)], average_loss:0.2758, validation_loss:0.3294, val_accuracy:0.9101
Epoch [109/range(100, 125)], average_loss:0.2736, validation_loss:0.3655, val_accuracy:0.8315
Epoch [110/range(100, 125)], average_loss:0.2735, validation_loss:0.3592, val_accuracy:0.8652
Epoch [111/range(100, 125)], average_loss:0.2732, validation

In [166]:
with torch.no_grad():
    x_pred = np.argmax(m9.model(torch.tensor(X_test).float().to(m9.device)).cpu().detach(), axis=1)


In [167]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [168]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8380798067310606

In [169]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8971677820267686

In [170]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3619,  400,   43,  173],
       [ 378, 9800,   48,  219],
       [  11,   56,  391,   21],
       [ 217,  141,   14, 1205]], dtype=int64)

In [171]:
#torch.save(m9.model, "models_and_losses/CNN-full_conv_lr-1e-3_bs-100_100epochs.pt")

In [173]:
with torch.no_grad():
    x_pred = np.argmax(m9.model(torch.tensor(X_test).float().to(m9.device)).cpu().detach(), axis=1)


In [174]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [175]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8355944835900052

In [176]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8970482791586998

In [177]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3566,  470,   25,  174],
       [ 341, 9907,   38,  159],
       [  26,   73,  373,    7],
       [ 205,  180,   25, 1167]], dtype=int64)

In [178]:
#torch.save(m9.model, "models_and_losses/CNN-full_conv_lr-1e-3_bs-100_125epochs.pt")

# big Batch
### model 10 without scheduler

In [213]:
with torch.no_grad():
    x_pred = np.argmax(m10.model(torch.tensor(X_test).float().to(m10.device)).cpu().detach(), axis=1)


In [214]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [215]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8632828120580631

In [216]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9130019120458891

In [217]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3707,  394,   28,  106],
       [ 318, 9990,   35,  102],
       [  15,   46,  400,   18],
       [ 240,  143,   11, 1183]], dtype=int64)

### with scheduler

In [219]:
CNN_10 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [220]:
optimizer= torch.optim.Adam(CNN_10.parameters(), lr=1e-3)

In [221]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=300, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=300, shuffle=True)

In [222]:
m10 = Trainer(
        model=CNN_10,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_lr-{1e-3,1e-4}_bs-300",
        verbose=True
        )

cuda:0


In [223]:
lh = m10(epochs=75, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/range(0, 75)], average_loss:0.5369, validation_loss:0.4355, val_accuracy:0.8413
Epoch [2/range(0, 75)], average_loss:0.3980, validation_loss:0.4019, val_accuracy:0.8466
Epoch [3/range(0, 75)], average_loss:0.3553, validation_loss:0.3661, val_accuracy:0.8836
Epoch [4/range(0, 75)], average_loss:0.3341, validation_loss:0.3237, val_accuracy:0.8995
Epoch [5/range(0, 75)], average_loss:0.3153, validation_loss:0.3106, val_accuracy:0.9101
Epoch [6/range(0, 75)], average_loss:0.3026, validation_loss:0.3037, val_accuracy:0.8995
Epoch [7/range(0, 75)], average_loss:0.2903, validation_loss:0.2971, val_accuracy:0.8942
Epoch [8/range(0, 75)], average_loss:0.2920, validation_loss:0.3129, val_accuracy:0.8466
Epoch [9/range(0, 75)], average_loss:0.2825, validation_loss:0.2868, val_accuracy:0.9206
Epoch [10/range(0, 75)], average_loss:0.2722, validation_loss:0.2818, val_accuracy:0.9048
Epoch [11/range(0, 75)], average_loss:0.2530, validation_loss:0.2674, val_accuracy:0.8677
Epoch [12/range(0, 

In [231]:
lh = m10(epochs=range(75,100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [76/range(75, 100)], average_loss:0.2110, validation_loss:0.2449, val_accuracy:0.8995
Epoch [77/range(75, 100)], average_loss:0.2132, validation_loss:0.2475, val_accuracy:0.9312
Epoch [78/range(75, 100)], average_loss:0.2115, validation_loss:0.2398, val_accuracy:0.8889
Epoch [79/range(75, 100)], average_loss:0.2080, validation_loss:0.2429, val_accuracy:0.8730
Epoch [80/range(75, 100)], average_loss:0.2082, validation_loss:0.2478, val_accuracy:0.8995
Epoch [81/range(75, 100)], average_loss:0.2090, validation_loss:0.2456, val_accuracy:0.9471
Epoch [82/range(75, 100)], average_loss:0.2069, validation_loss:0.2450, val_accuracy:0.8889
Epoch [83/range(75, 100)], average_loss:0.2079, validation_loss:0.2477, val_accuracy:0.9471
Epoch [84/range(75, 100)], average_loss:0.2069, validation_loss:0.2439, val_accuracy:0.8889
Epoch [85/range(75, 100)], average_loss:0.2072, validation_loss:0.2424, val_accuracy:0.9206
Epoch [86/range(75, 100)], average_loss:0.2055, validation_loss:0.2371, val_accu

In [238]:
lh = m10(epochs=range(100,120), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/range(100, 120)], average_loss:0.1991, validation_loss:0.2411, val_accuracy:0.9048
Epoch [102/range(100, 120)], average_loss:0.2008, validation_loss:0.2394, val_accuracy:0.8519
Epoch [103/range(100, 120)], average_loss:0.1991, validation_loss:0.2361, val_accuracy:0.9153
Epoch [104/range(100, 120)], average_loss:0.1991, validation_loss:0.2489, val_accuracy:0.9418
Epoch [105/range(100, 120)], average_loss:0.2012, validation_loss:0.2416, val_accuracy:0.9206
Epoch [106/range(100, 120)], average_loss:0.1978, validation_loss:0.2395, val_accuracy:0.9206
Epoch [107/range(100, 120)], average_loss:0.1969, validation_loss:0.2397, val_accuracy:0.9048
Epoch [108/range(100, 120)], average_loss:0.1982, validation_loss:0.2376, val_accuracy:0.8942
Epoch [109/range(100, 120)], average_loss:0.1990, validation_loss:0.2430, val_accuracy:0.8942
Epoch [110/range(100, 120)], average_loss:0.1979, validation_loss:0.2340, val_accuracy:0.9259
Epoch [111/range(100, 120)], average_loss:0.1970, validation

In [245]:
lh = m10(epochs=range(120,140), train_loader=train_loader, validation_loader=valid_loader)

Epoch [121/range(120, 140)], average_loss:0.1928, validation_loss:0.2370, val_accuracy:0.9153
Epoch [122/range(120, 140)], average_loss:0.1928, validation_loss:0.2398, val_accuracy:0.9153
Epoch [123/range(120, 140)], average_loss:0.1938, validation_loss:0.2399, val_accuracy:0.9206
Epoch [124/range(120, 140)], average_loss:0.1929, validation_loss:0.2360, val_accuracy:0.9312
Epoch [125/range(120, 140)], average_loss:0.1945, validation_loss:0.2401, val_accuracy:0.9365
Epoch [126/range(120, 140)], average_loss:0.1907, validation_loss:0.2339, val_accuracy:0.9206
Epoch [127/range(120, 140)], average_loss:0.1899, validation_loss:0.2389, val_accuracy:0.9153
Epoch [128/range(120, 140)], average_loss:0.1919, validation_loss:0.2406, val_accuracy:0.8889
Epoch [129/range(120, 140)], average_loss:0.1905, validation_loss:0.2394, val_accuracy:0.9048
Epoch [130/range(120, 140)], average_loss:0.1907, validation_loss:0.2423, val_accuracy:0.8942
Epoch [131/range(120, 140)], average_loss:0.1883, validation

In [251]:
lh = m10(epochs=range(140,160), train_loader=train_loader, validation_loader=valid_loader)

Epoch [141/range(140, 160)], average_loss:0.1879, validation_loss:0.2362, val_accuracy:0.8836
Epoch [142/range(140, 160)], average_loss:0.1866, validation_loss:0.2461, val_accuracy:0.8995
Epoch [143/range(140, 160)], average_loss:0.1890, validation_loss:0.2363, val_accuracy:0.9365
Epoch [144/range(140, 160)], average_loss:0.1853, validation_loss:0.2414, val_accuracy:0.8942
Epoch [145/range(140, 160)], average_loss:0.1875, validation_loss:0.2412, val_accuracy:0.9153
Epoch [146/range(140, 160)], average_loss:0.1859, validation_loss:0.2411, val_accuracy:0.9524
Epoch [147/range(140, 160)], average_loss:0.1849, validation_loss:0.2403, val_accuracy:0.9101
Epoch [148/range(140, 160)], average_loss:0.1826, validation_loss:0.2459, val_accuracy:0.8836
Epoch [149/range(140, 160)], average_loss:0.1845, validation_loss:0.2373, val_accuracy:0.9048
Epoch [150/range(140, 160)], average_loss:0.1825, validation_loss:0.2340, val_accuracy:0.8624
Epoch [151/range(140, 160)], average_loss:0.1841, validation

In [184]:
m10.tb.close()

In [252]:
#m10.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m10.model(torch.tensor(X_test).float().to(m10.device)).cpu().detach(), axis=1)


In [253]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [254]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8786791784976762

In [255]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.923876673040153

In [256]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3805,   307,    23,   100],
       [  291, 10040,    20,    94],
       [   18,    43,   402,    16],
       [  228,   123,    11,  1215]], dtype=int64)

In [257]:
#torch.save(m10.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_lr-{1e-3,1e-4}_bs-300_epoch-160.pt")

In [263]:
m10.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m10.model(torch.tensor(X_test).float().to(m10.device)).cpu().detach(), axis=1)


In [264]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [265]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8888975901108741

In [266]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9321821223709369

In [267]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3892,   206,    27,   110],
       [  297, 10035,    27,    86],
       [   17,    29,   414,    19],
       [  215,    93,     9,  1260]], dtype=int64)

# Bigg batch size

In [12]:
CNN_11 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [13]:
optimizer= torch.optim.Adam(CNN_11.parameters(), lr=1e-3)

In [14]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [15]:
m11 = Trainer(
        model=CNN_11,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_lr-1e-3_bs-512",
        verbose=True
        )

cuda:0


In [16]:
lh = m11(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.5722, validation_loss:0.4303, val_accuracy:0.8312
Epoch [2/150], average_loss:0.3975, validation_loss:0.3891, val_accuracy:0.8961
Epoch [3/150], average_loss:0.3613, validation_loss:0.3532, val_accuracy:0.8312
Epoch [4/150], average_loss:0.3263, validation_loss:0.3263, val_accuracy:0.8831
Epoch [5/150], average_loss:0.3174, validation_loss:0.3158, val_accuracy:0.8571
Epoch [6/150], average_loss:0.2993, validation_loss:0.3021, val_accuracy:0.8442
Epoch [7/150], average_loss:0.2920, validation_loss:0.2896, val_accuracy:0.9351
Epoch [8/150], average_loss:0.2811, validation_loss:0.2910, val_accuracy:0.8961
Epoch [9/150], average_loss:0.2785, validation_loss:0.3023, val_accuracy:0.9221
Epoch [10/150], average_loss:0.2713, validation_loss:0.2988, val_accuracy:0.9221
Epoch [11/150], average_loss:0.2641, validation_loss:0.2785, val_accuracy:0.8442
Epoch [12/150], average_loss:0.2643, validation_loss:0.2723, val_accuracy:0.9481
Epoch [13/150], average_loss:0.2557, 

Epoch [103/150], average_loss:0.1439, validation_loss:0.2760, val_accuracy:0.8571
Epoch [104/150], average_loss:0.1435, validation_loss:0.2586, val_accuracy:0.9481
Epoch [105/150], average_loss:0.1458, validation_loss:0.2612, val_accuracy:0.8831
Epoch [106/150], average_loss:0.1436, validation_loss:0.2691, val_accuracy:0.8831
Epoch [107/150], average_loss:0.1425, validation_loss:0.2778, val_accuracy:0.8701
Epoch [108/150], average_loss:0.1441, validation_loss:0.2555, val_accuracy:0.9351
Epoch [109/150], average_loss:0.1423, validation_loss:0.2756, val_accuracy:0.9091
Epoch [110/150], average_loss:0.1446, validation_loss:0.2644, val_accuracy:0.8831
Epoch [111/150], average_loss:0.1369, validation_loss:0.2707, val_accuracy:0.8831
Epoch [112/150], average_loss:0.1394, validation_loss:0.2728, val_accuracy:0.8571
Epoch [113/150], average_loss:0.1372, validation_loss:0.2542, val_accuracy:0.9610
Epoch [114/150], average_loss:0.1375, validation_loss:0.2701, val_accuracy:0.9221
Epoch [115/150],

# scheduler

In [16]:
CNN_12 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [17]:
optimizer12 = torch.optim.Adam(CNN_12.parameters(), lr=1e-3)

In [18]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [19]:
m12 = Trainer(
        model=CNN_12,
        optimizer=optimizer12,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer12, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [20]:
lh = m12(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.6074, validation_loss:0.4843, val_accuracy:0.7532
Epoch [2/150], average_loss:0.4492, validation_loss:0.4186, val_accuracy:0.7662
Epoch [3/150], average_loss:0.3991, validation_loss:0.3715, val_accuracy:0.8571
Epoch [4/150], average_loss:0.3557, validation_loss:0.3382, val_accuracy:0.8312
Epoch [5/150], average_loss:0.3373, validation_loss:0.3291, val_accuracy:0.8312
Epoch [6/150], average_loss:0.3216, validation_loss:0.3284, val_accuracy:0.7922
Epoch [7/150], average_loss:0.3071, validation_loss:0.3191, val_accuracy:0.9481
Epoch [8/150], average_loss:0.3054, validation_loss:0.2953, val_accuracy:0.8961
Epoch [9/150], average_loss:0.2908, validation_loss:0.3100, val_accuracy:0.9091
Epoch [10/150], average_loss:0.2860, validation_loss:0.2998, val_accuracy:0.7922
Epoch [11/150], average_loss:0.2855, validation_loss:0.3252, val_accuracy:0.8312
Epoch [12/150], average_loss:0.2747, validation_loss:0.2926, val_accuracy:0.8571
Epoch [13/150], average_loss:0.2735, 

Epoch [103/150], average_loss:0.1903, validation_loss:0.2420, val_accuracy:0.8701
Epoch [104/150], average_loss:0.1924, validation_loss:0.2262, val_accuracy:0.9091
Epoch [105/150], average_loss:0.1911, validation_loss:0.2402, val_accuracy:0.8961
Epoch [106/150], average_loss:0.1919, validation_loss:0.2419, val_accuracy:0.8831
Epoch [107/150], average_loss:0.1911, validation_loss:0.2353, val_accuracy:0.8961
Epoch [108/150], average_loss:0.1917, validation_loss:0.2366, val_accuracy:0.8831
Epoch [109/150], average_loss:0.1913, validation_loss:0.2326, val_accuracy:0.9221
Epoch [110/150], average_loss:0.1926, validation_loss:0.2397, val_accuracy:0.8701
Epoch [111/150], average_loss:0.1892, validation_loss:0.2280, val_accuracy:0.9481
Epoch [112/150], average_loss:0.1903, validation_loss:0.2307, val_accuracy:0.9221
Epoch [113/150], average_loss:0.1883, validation_loss:0.2317, val_accuracy:0.8961
Epoch [114/150], average_loss:0.1935, validation_loss:0.2375, val_accuracy:0.9091
Epoch [115/150],

In [37]:
m12.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m12.model(torch.tensor(X_test).float().to(m12.device)).cpu().detach(), axis=1)


In [38]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [39]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8903114917054931

In [40]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9302700764818356

In [41]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3957,  139,   25,  114],
       [ 411, 9917,   30,   87],
       [  15,   28,  420,   16],
       [ 206,   86,   10, 1275]], dtype=int64)

In [42]:
#torch.save(m12.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_lr-{1e-3,1e-4}_bs-512.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Scheduler 0.001 -> 0.00001

In [21]:
CNN_13 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ELU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ELU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ELU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [22]:
optimizer13= torch.optim.Adam(CNN_13.parameters(), lr=1e-3)

In [23]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [24]:
m13 = Trainer(
        model=CNN_13,
        optimizer=optimizer13,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer13, milestones=[20,80], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_ELU_lr-{1e-3,1e-4,1e-5}_bs-512",
        verbose=True
        )

cuda:0


In [25]:
lh = m13(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.6481, validation_loss:0.5606, val_accuracy:0.7662
Epoch [2/150], average_loss:0.5301, validation_loss:0.5090, val_accuracy:0.7792
Epoch [3/150], average_loss:0.5043, validation_loss:0.4932, val_accuracy:0.7922
Epoch [4/150], average_loss:0.4834, validation_loss:0.4685, val_accuracy:0.8961
Epoch [5/150], average_loss:0.4631, validation_loss:0.4772, val_accuracy:0.8312
Epoch [6/150], average_loss:0.4467, validation_loss:0.4401, val_accuracy:0.8442
Epoch [7/150], average_loss:0.4336, validation_loss:0.4442, val_accuracy:0.7922
Epoch [8/150], average_loss:0.4159, validation_loss:0.4197, val_accuracy:0.7792
Epoch [9/150], average_loss:0.3907, validation_loss:0.4080, val_accuracy:0.8961
Epoch [10/150], average_loss:0.3770, validation_loss:0.3927, val_accuracy:0.8831
Epoch [11/150], average_loss:0.3642, validation_loss:0.3736, val_accuracy:0.8571
Epoch [12/150], average_loss:0.3564, validation_loss:0.3712, val_accuracy:0.8052
Epoch [13/150], average_loss:0.3569, 

Epoch [103/150], average_loss:0.2586, validation_loss:0.3028, val_accuracy:0.8312
Epoch [104/150], average_loss:0.2576, validation_loss:0.3030, val_accuracy:0.8442
Epoch [105/150], average_loss:0.2582, validation_loss:0.2954, val_accuracy:0.8312
Epoch [106/150], average_loss:0.2560, validation_loss:0.2999, val_accuracy:0.8442
Epoch [107/150], average_loss:0.2574, validation_loss:0.2981, val_accuracy:0.9610
Epoch [108/150], average_loss:0.2564, validation_loss:0.3040, val_accuracy:0.8442
Epoch [109/150], average_loss:0.2586, validation_loss:0.3031, val_accuracy:0.8701
Epoch [110/150], average_loss:0.2564, validation_loss:0.2958, val_accuracy:0.9091
Epoch [111/150], average_loss:0.2580, validation_loss:0.2945, val_accuracy:0.9481
Epoch [112/150], average_loss:0.2549, validation_loss:0.2995, val_accuracy:0.8442
Epoch [113/150], average_loss:0.2568, validation_loss:0.2986, val_accuracy:0.8571
Epoch [114/150], average_loss:0.2596, validation_loss:0.2930, val_accuracy:0.9481
Epoch [115/150],

In [26]:
m13.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m13.model(torch.tensor(X_test).float().to(m13.device)).cpu().detach(), axis=1)


In [27]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [28]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8702245340803618

In [29]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9103728489483748

In [30]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3664,  416,   31,  124],
       [ 400, 9908,   25,  112],
       [  17,   34,  413,   15],
       [ 215,  100,   11, 1251]], dtype=int64)

In [31]:
torch.save(m13.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_ELU_lr-{1e-3,1e-4,1e-5}_bs-512.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ELU. It won't be checked for correctness upon loa

# scheduler 0.001->0.00001 RELU

In [9]:
CNN_14 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [10]:
optimizer14= torch.optim.Adam(CNN_14.parameters(), lr=1e-3)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [12]:
m14 = Trainer(
        model=CNN_14,
        optimizer=optimizer14,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer14, milestones=[20,80], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4,1e-5}_bs-512",
        verbose=True
        )

cuda:0


In [13]:
lh = m14(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.5898, validation_loss:0.4605, val_accuracy:0.7403
Epoch [2/150], average_loss:0.4464, validation_loss:0.3857, val_accuracy:0.8961
Epoch [3/150], average_loss:0.3963, validation_loss:0.3536, val_accuracy:0.9221
Epoch [4/150], average_loss:0.3706, validation_loss:0.3382, val_accuracy:0.8701
Epoch [5/150], average_loss:0.3485, validation_loss:0.3179, val_accuracy:0.8442
Epoch [6/150], average_loss:0.3273, validation_loss:0.3709, val_accuracy:0.8701
Epoch [7/150], average_loss:0.3222, validation_loss:0.2881, val_accuracy:0.8701
Epoch [8/150], average_loss:0.3179, validation_loss:0.2865, val_accuracy:0.9091
Epoch [9/150], average_loss:0.3031, validation_loss:0.2757, val_accuracy:0.8571
Epoch [10/150], average_loss:0.2944, validation_loss:0.2782, val_accuracy:0.9221
Epoch [11/150], average_loss:0.2950, validation_loss:0.2746, val_accuracy:0.8961
Epoch [12/150], average_loss:0.2872, validation_loss:0.2769, val_accuracy:0.9221
Epoch [13/150], average_loss:0.2858, 

Epoch [103/150], average_loss:0.1991, validation_loss:0.2058, val_accuracy:0.9091
Epoch [104/150], average_loss:0.1985, validation_loss:0.2041, val_accuracy:0.8961
Epoch [105/150], average_loss:0.1999, validation_loss:0.2066, val_accuracy:0.9221
Epoch [106/150], average_loss:0.2014, validation_loss:0.2030, val_accuracy:0.9481
Epoch [107/150], average_loss:0.2009, validation_loss:0.2050, val_accuracy:0.9221
Epoch [108/150], average_loss:0.1997, validation_loss:0.2057, val_accuracy:0.8701
Epoch [109/150], average_loss:0.2020, validation_loss:0.2146, val_accuracy:0.8831
Epoch [110/150], average_loss:0.2001, validation_loss:0.2081, val_accuracy:0.8831
Epoch [111/150], average_loss:0.2005, validation_loss:0.2021, val_accuracy:0.9610
Epoch [112/150], average_loss:0.2008, validation_loss:0.2015, val_accuracy:0.9351
Epoch [113/150], average_loss:0.2014, validation_loss:0.2029, val_accuracy:0.9221
Epoch [114/150], average_loss:0.2004, validation_loss:0.2024, val_accuracy:0.9351
Epoch [115/150],

In [14]:
m14.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m14.model(torch.tensor(X_test).float().to(m14.device)).cpu().detach(), axis=1)


In [15]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [16]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8893648716761628

In [17]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9311065965583174

In [18]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3912,  200,   26,   97],
       [ 334, 9998,   29,   84],
       [  16,   31,  417,   15],
       [ 224,   87,   10, 1256]], dtype=int64)

In [20]:
#torch.save(m14.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4,1e-5}_bs-512.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon lo

# Retrain model 12

In [21]:
CNN_15 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [22]:
optimizer15= torch.optim.Adam(CNN_15.parameters(), lr=1e-3)

In [23]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [24]:
m15 = Trainer(
        model=CNN_15,
        optimizer=optimizer15,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer15, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [25]:
lh = m15(epochs=200, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/200], average_loss:0.6071, validation_loss:0.4563, val_accuracy:0.8052
Epoch [2/200], average_loss:0.4437, validation_loss:0.3644, val_accuracy:0.8571
Epoch [3/200], average_loss:0.3947, validation_loss:0.3306, val_accuracy:0.8701
Epoch [4/200], average_loss:0.3581, validation_loss:0.3570, val_accuracy:0.8831
Epoch [5/200], average_loss:0.3340, validation_loss:0.3702, val_accuracy:0.9351
Epoch [6/200], average_loss:0.3237, validation_loss:0.2840, val_accuracy:0.8831
Epoch [7/200], average_loss:0.3064, validation_loss:0.2709, val_accuracy:0.8961
Epoch [8/200], average_loss:0.3040, validation_loss:0.2799, val_accuracy:0.8831
Epoch [9/200], average_loss:0.2935, validation_loss:0.2877, val_accuracy:0.8831
Epoch [10/200], average_loss:0.2888, validation_loss:0.2802, val_accuracy:0.9221
Epoch [11/200], average_loss:0.2860, validation_loss:0.2651, val_accuracy:0.8831
Epoch [12/200], average_loss:0.2747, validation_loss:0.2604, val_accuracy:0.8701
Epoch [13/200], average_loss:0.2715, 

Epoch [103/200], average_loss:0.1976, validation_loss:0.2023, val_accuracy:0.9221
Epoch [104/200], average_loss:0.1980, validation_loss:0.2044, val_accuracy:0.9091
Epoch [105/200], average_loss:0.1969, validation_loss:0.2107, val_accuracy:0.8701
Epoch [106/200], average_loss:0.1970, validation_loss:0.1954, val_accuracy:0.9610
Epoch [107/200], average_loss:0.1943, validation_loss:0.2038, val_accuracy:0.8961
Epoch [108/200], average_loss:0.1978, validation_loss:0.2042, val_accuracy:0.8831
Epoch [109/200], average_loss:0.1959, validation_loss:0.2108, val_accuracy:0.8831
Epoch [110/200], average_loss:0.1947, validation_loss:0.2037, val_accuracy:0.8961
Epoch [111/200], average_loss:0.1945, validation_loss:0.2213, val_accuracy:0.9351
Epoch [112/200], average_loss:0.1938, validation_loss:0.1999, val_accuracy:0.9481
Epoch [113/200], average_loss:0.1938, validation_loss:0.2100, val_accuracy:0.8701
Epoch [114/200], average_loss:0.1947, validation_loss:0.2006, val_accuracy:0.9091
Epoch [115/200],

In [26]:
m15.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m15.model(torch.tensor(X_test).float().to(m15.device)).cpu().detach(), axis=1)


In [27]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [28]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8906257967458273

In [29]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.933914913957935

In [30]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3942,   147,    30,   116],
       [  321, 10002,    28,    94],
       [   14,    32,   416,    17],
       [  210,    90,     7,  1270]], dtype=int64)

In [31]:
#torch.save(m15.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epochs_200_Jan30_13-36-38.pt")

In [32]:
lh = m15(epochs=range(200,250), train_loader=train_loader, validation_loader=valid_loader)

Epoch [201/250], average_loss:0.1743, validation_loss:0.1956, val_accuracy:0.8961
Epoch [202/250], average_loss:0.1720, validation_loss:0.1987, val_accuracy:0.9091
Epoch [203/250], average_loss:0.1731, validation_loss:0.1949, val_accuracy:0.9481
Epoch [204/250], average_loss:0.1722, validation_loss:0.1983, val_accuracy:0.9091
Epoch [205/250], average_loss:0.1725, validation_loss:0.1986, val_accuracy:0.9091
Epoch [206/250], average_loss:0.1721, validation_loss:0.1934, val_accuracy:0.9221
Epoch [207/250], average_loss:0.1712, validation_loss:0.1981, val_accuracy:0.9610
Epoch [208/250], average_loss:0.1699, validation_loss:0.1913, val_accuracy:0.9740
Epoch [209/250], average_loss:0.1700, validation_loss:0.1894, val_accuracy:0.9610
Epoch [210/250], average_loss:0.1698, validation_loss:0.1977, val_accuracy:0.9221
Epoch [211/250], average_loss:0.1705, validation_loss:0.2088, val_accuracy:0.8961
Epoch [212/250], average_loss:0.1723, validation_loss:0.2033, val_accuracy:0.9221
Epoch [213/250],

In [33]:
m15.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m15.model(torch.tensor(X_test).float().to(m15.device)).cpu().detach(), axis=1)


In [34]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [35]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8896463543673009

In [36]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9343331739961759

In [37]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3928,   175,    29,   103],
       [  283, 10032,    35,    95],
       [   16,    26,   418,    19],
       [  218,    92,     8,  1259]], dtype=int64)

In [38]:
#torch.save(m15.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epochs_250_Jan30_13-36-38.pt")

# last test with elu

In [39]:
CNN_16 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ELU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ELU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ELU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Linear(in_features=64*18, out_features=256),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=256, out_features=4)
)

In [40]:
optimizer16= torch.optim.Adam(CNN_16.parameters(), lr=1e-3)

In [41]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [42]:
m16 = Trainer(
        model=CNN_16,
        optimizer=optimizer16,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer16, milestones=[25], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.III-Conv1-BN-Drop-0.3_ELU_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [43]:
lh = m16(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.6387, validation_loss:0.5153, val_accuracy:0.7792
Epoch [2/150], average_loss:0.5256, validation_loss:0.4974, val_accuracy:0.8052
Epoch [3/150], average_loss:0.4925, validation_loss:0.4704, val_accuracy:0.7662
Epoch [4/150], average_loss:0.4842, validation_loss:0.4385, val_accuracy:0.7532
Epoch [5/150], average_loss:0.4582, validation_loss:0.4412, val_accuracy:0.7792
Epoch [6/150], average_loss:0.4367, validation_loss:0.4524, val_accuracy:0.8052
Epoch [7/150], average_loss:0.4121, validation_loss:0.3940, val_accuracy:0.8701
Epoch [8/150], average_loss:0.3957, validation_loss:0.3731, val_accuracy:0.8701
Epoch [9/150], average_loss:0.3841, validation_loss:0.3613, val_accuracy:0.8961
Epoch [10/150], average_loss:0.3570, validation_loss:0.3326, val_accuracy:0.8442
Epoch [11/150], average_loss:0.3446, validation_loss:0.3095, val_accuracy:0.8831
Epoch [12/150], average_loss:0.3380, validation_loss:0.3369, val_accuracy:0.8571
Epoch [13/150], average_loss:0.3365, 

Epoch [103/150], average_loss:0.2215, validation_loss:0.2338, val_accuracy:0.9481
Epoch [104/150], average_loss:0.2218, validation_loss:0.2350, val_accuracy:0.8701
Epoch [105/150], average_loss:0.2268, validation_loss:0.2397, val_accuracy:0.9221
Epoch [106/150], average_loss:0.2238, validation_loss:0.2410, val_accuracy:0.8701
Epoch [107/150], average_loss:0.2235, validation_loss:0.2396, val_accuracy:0.8831
Epoch [108/150], average_loss:0.2224, validation_loss:0.2400, val_accuracy:0.8961
Epoch [109/150], average_loss:0.2198, validation_loss:0.2427, val_accuracy:0.8961
Epoch [110/150], average_loss:0.2219, validation_loss:0.2388, val_accuracy:0.9091
Epoch [111/150], average_loss:0.2207, validation_loss:0.2375, val_accuracy:0.9091
Epoch [112/150], average_loss:0.2221, validation_loss:0.2381, val_accuracy:0.9221
Epoch [113/150], average_loss:0.2234, validation_loss:0.2416, val_accuracy:0.8961
Epoch [114/150], average_loss:0.2198, validation_loss:0.2328, val_accuracy:0.9610
Epoch [115/150],

In [44]:
m16.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m16.model(torch.tensor(X_test).float().to(m16.device)).cpu().detach(), axis=1)


In [45]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [46]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8742859351404654

In [47]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9184990439770554

In [48]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3808,  265,   25,  137],
       [ 414, 9883,   35,  113],
       [  22,   34,  404,   19],
       [ 187,  102,   11, 1277]], dtype=int64)

In [49]:
lh = m16(epochs=range(150,250), train_loader=train_loader, validation_loader=valid_loader)

Epoch [151/250], average_loss:0.2070, validation_loss:0.2352, val_accuracy:0.8961
Epoch [152/250], average_loss:0.2107, validation_loss:0.2405, val_accuracy:0.8571
Epoch [153/250], average_loss:0.2085, validation_loss:0.2293, val_accuracy:0.9221
Epoch [154/250], average_loss:0.2093, validation_loss:0.2256, val_accuracy:0.9481
Epoch [155/250], average_loss:0.2064, validation_loss:0.2303, val_accuracy:0.9351
Epoch [156/250], average_loss:0.2055, validation_loss:0.2318, val_accuracy:0.9091
Epoch [157/250], average_loss:0.2063, validation_loss:0.2296, val_accuracy:0.9221
Epoch [158/250], average_loss:0.2064, validation_loss:0.2278, val_accuracy:0.9221
Epoch [159/250], average_loss:0.2068, validation_loss:0.2303, val_accuracy:0.8831
Epoch [160/250], average_loss:0.2055, validation_loss:0.2307, val_accuracy:0.9221
Epoch [161/250], average_loss:0.2055, validation_loss:0.2346, val_accuracy:0.9481
Epoch [162/250], average_loss:0.2049, validation_loss:0.2288, val_accuracy:0.9481
Epoch [163/250],

In [50]:
m16.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m16.model(torch.tensor(X_test).float().to(m16.device)).cpu().detach(), axis=1)


In [51]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [52]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8776778430454325

In [53]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9223828871892925

In [54]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3821,  255,   27,  132],
       [ 357, 9945,   32,  111],
       [  21,   31,  408,   19],
       [ 205,  100,    9, 1263]], dtype=int64)

In [55]:
#torch.save(m16.model, "models_and_losses/CNN-m.III-Conv1-BN-Drop-0.3_ELU_lr-{1e-3,1e-4}_bs-512_epochs_250_Jan30_14-37-36.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ELU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [56]:
lh = m16(epochs=range(250,350), train_loader=train_loader, validation_loader=valid_loader)

Epoch [251/350], average_loss:0.1829, validation_loss:0.2401, val_accuracy:0.9351
Epoch [252/350], average_loss:0.1858, validation_loss:0.2324, val_accuracy:0.9481
Epoch [253/350], average_loss:0.1843, validation_loss:0.2382, val_accuracy:0.9091
Epoch [254/350], average_loss:0.1854, validation_loss:0.2343, val_accuracy:0.9091
Epoch [255/350], average_loss:0.1823, validation_loss:0.2367, val_accuracy:0.9481
Epoch [256/350], average_loss:0.1864, validation_loss:0.2375, val_accuracy:0.8701
Epoch [257/350], average_loss:0.1839, validation_loss:0.2348, val_accuracy:0.9221
Epoch [258/350], average_loss:0.1838, validation_loss:0.2359, val_accuracy:0.9221
Epoch [259/350], average_loss:0.1837, validation_loss:0.2334, val_accuracy:0.9481
Epoch [260/350], average_loss:0.1825, validation_loss:0.2358, val_accuracy:0.9091
Epoch [261/350], average_loss:0.1823, validation_loss:0.2377, val_accuracy:0.9091
Epoch [262/350], average_loss:0.1824, validation_loss:0.2300, val_accuracy:0.9351
Epoch [263/350],

In [57]:
m16.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m16.model(torch.tensor(X_test).float().to(m16.device)).cpu().detach(), axis=1)


In [58]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [59]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8757419907129813

In [60]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9223828871892925

In [61]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3730,   345,    28,   132],
       [  257, 10045,    35,   108],
       [   23,    29,   407,    20],
       [  207,   107,     8,  1255]], dtype=int64)

# Full Conv

In [9]:
CNN_17 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0, bias=False), # n*1*160 -> n*32*156
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3), #n*32*156 -> n*32*52
    
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0, bias=False),# n*32*52->n*64*48
            nn.ReLU(inplace=True),
    
            layers.Flatten(out_features=48*64),
            nn.Linear(in_features=48*64, out_features=4)
)

In [10]:
optimizer17= torch.optim.Adam(CNN_17.parameters(), lr=1e-3)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [12]:
m17 = Trainer(
        model=CNN_17,
        optimizer=optimizer17,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer17, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-full_conv_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [13]:
lh = m17(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.8303, validation_loss:0.7244, val_accuracy:0.6753
Epoch [2/150], average_loss:0.6474, validation_loss:0.5974, val_accuracy:0.7922
Epoch [3/150], average_loss:0.5661, validation_loss:0.5391, val_accuracy:0.7922
Epoch [4/150], average_loss:0.5204, validation_loss:0.5023, val_accuracy:0.8442
Epoch [5/150], average_loss:0.4975, validation_loss:0.4832, val_accuracy:0.8961
Epoch [6/150], average_loss:0.4753, validation_loss:0.4790, val_accuracy:0.7922
Epoch [7/150], average_loss:0.4629, validation_loss:0.4692, val_accuracy:0.8571
Epoch [8/150], average_loss:0.4609, validation_loss:0.4641, val_accuracy:0.8312
Epoch [9/150], average_loss:0.4574, validation_loss:0.4450, val_accuracy:0.9091
Epoch [10/150], average_loss:0.4429, validation_loss:0.4575, val_accuracy:0.8182
Epoch [11/150], average_loss:0.4356, validation_loss:0.4391, val_accuracy:0.8961
Epoch [12/150], average_loss:0.4342, validation_loss:0.4501, val_accuracy:0.7143
Epoch [13/150], average_loss:0.4268, 

Epoch [103/150], average_loss:0.2651, validation_loss:0.3135, val_accuracy:0.8442
Epoch [104/150], average_loss:0.2651, validation_loss:0.3042, val_accuracy:0.9351
Epoch [105/150], average_loss:0.2648, validation_loss:0.3101, val_accuracy:0.8442
Epoch [106/150], average_loss:0.2645, validation_loss:0.3056, val_accuracy:0.8961
Epoch [107/150], average_loss:0.2649, validation_loss:0.3075, val_accuracy:0.8312
Epoch [108/150], average_loss:0.2651, validation_loss:0.3054, val_accuracy:0.9221
Epoch [109/150], average_loss:0.2646, validation_loss:0.3090, val_accuracy:0.8182
Epoch [110/150], average_loss:0.2643, validation_loss:0.3059, val_accuracy:0.8701
Epoch [111/150], average_loss:0.2647, validation_loss:0.3050, val_accuracy:0.8961
Epoch [112/150], average_loss:0.2639, validation_loss:0.3112, val_accuracy:0.8442
Epoch [113/150], average_loss:0.2643, validation_loss:0.3043, val_accuracy:0.9351
Epoch [114/150], average_loss:0.2640, validation_loss:0.3122, val_accuracy:0.8442
Epoch [115/150],

In [14]:
lh = m17(epochs=range(150,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [151/200], average_loss:0.2606, validation_loss:0.3074, val_accuracy:0.8701
Epoch [152/200], average_loss:0.2605, validation_loss:0.3098, val_accuracy:0.8571
Epoch [153/200], average_loss:0.2599, validation_loss:0.3048, val_accuracy:0.9091
Epoch [154/200], average_loss:0.2592, validation_loss:0.3054, val_accuracy:0.9091
Epoch [155/200], average_loss:0.2603, validation_loss:0.3044, val_accuracy:0.8442
Epoch [156/200], average_loss:0.2595, validation_loss:0.3011, val_accuracy:0.9481
Epoch [157/200], average_loss:0.2596, validation_loss:0.3039, val_accuracy:0.8571
Epoch [158/200], average_loss:0.2601, validation_loss:0.3074, val_accuracy:0.8442
Epoch [159/200], average_loss:0.2593, validation_loss:0.3055, val_accuracy:0.9091
Epoch [160/200], average_loss:0.2594, validation_loss:0.3057, val_accuracy:0.8831
Epoch [161/200], average_loss:0.2593, validation_loss:0.3026, val_accuracy:0.8961
Epoch [162/200], average_loss:0.2590, validation_loss:0.3087, val_accuracy:0.8831
Epoch [163/200],

In [15]:
m17.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m17.model(torch.tensor(X_test).float().to(m17.device)).cpu().detach(), axis=1)


In [16]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [17]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8487120996854254

In [18]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9009918738049714

In [19]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3610,  491,   28,  106],
       [ 342, 9927,   28,  148],
       [  30,   45,  389,   15],
       [ 240,  171,   13, 1153]], dtype=int64)

In [21]:
#torch.save(m17.model, "models_and_losses/CNN-full_conv_lr-{1e-3,1e-4}_bs-512.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon lo

In [22]:
m17.tb.close()

## Full conv m.II

In [23]:
CNN_18 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*32*78
            nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=3), #n*32*156 -> n*32*52
    
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=6, stride=2, padding=0, bias=False),# n*32*78->n*64*37
            nn.ReLU(inplace=True),
    
            layers.Flatten(out_features=37*64),
            nn.Linear(in_features=37*64, out_features=4)
)

In [24]:
optimizer18= torch.optim.Adam(CNN_18.parameters(), lr=1e-3)

In [25]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [26]:
m18 = Trainer(
        model=CNN_18,
        optimizer=optimizer18,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer18, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="CNN-full_conv_m.II_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [27]:
lh = m18(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.8400, validation_loss:0.6971, val_accuracy:0.7403
Epoch [2/150], average_loss:0.6439, validation_loss:0.5874, val_accuracy:0.8442
Epoch [3/150], average_loss:0.5813, validation_loss:0.5626, val_accuracy:0.8312
Epoch [4/150], average_loss:0.5452, validation_loss:0.5364, val_accuracy:0.7792
Epoch [5/150], average_loss:0.5236, validation_loss:0.5263, val_accuracy:0.8182
Epoch [6/150], average_loss:0.5106, validation_loss:0.5031, val_accuracy:0.8571
Epoch [7/150], average_loss:0.4905, validation_loss:0.5130, val_accuracy:0.8442
Epoch [8/150], average_loss:0.4747, validation_loss:0.4836, val_accuracy:0.7143
Epoch [9/150], average_loss:0.4672, validation_loss:0.4644, val_accuracy:0.8312
Epoch [10/150], average_loss:0.4603, validation_loss:0.4612, val_accuracy:0.8571
Epoch [11/150], average_loss:0.4514, validation_loss:0.4481, val_accuracy:0.8571
Epoch [12/150], average_loss:0.4401, validation_loss:0.4443, val_accuracy:0.8831
Epoch [13/150], average_loss:0.4392, 

Epoch [103/150], average_loss:0.2670, validation_loss:0.3060, val_accuracy:0.8831
Epoch [104/150], average_loss:0.2670, validation_loss:0.3056, val_accuracy:0.8831
Epoch [105/150], average_loss:0.2668, validation_loss:0.3055, val_accuracy:0.8442
Epoch [106/150], average_loss:0.2666, validation_loss:0.3137, val_accuracy:0.9091
Epoch [107/150], average_loss:0.2664, validation_loss:0.3026, val_accuracy:0.9091
Epoch [108/150], average_loss:0.2663, validation_loss:0.3035, val_accuracy:0.9221
Epoch [109/150], average_loss:0.2666, validation_loss:0.3020, val_accuracy:0.9351
Epoch [110/150], average_loss:0.2667, validation_loss:0.3024, val_accuracy:0.8961
Epoch [111/150], average_loss:0.2667, validation_loss:0.3057, val_accuracy:0.8831
Epoch [112/150], average_loss:0.2661, validation_loss:0.3053, val_accuracy:0.8442
Epoch [113/150], average_loss:0.2664, validation_loss:0.3011, val_accuracy:0.9221
Epoch [114/150], average_loss:0.2658, validation_loss:0.3004, val_accuracy:0.8961
Epoch [115/150],

In [33]:
lh = m18(epochs=range(150,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [151/200], average_loss:0.2616, validation_loss:0.2972, val_accuracy:0.8831
Epoch [152/200], average_loss:0.2613, validation_loss:0.2964, val_accuracy:0.9610
Epoch [153/200], average_loss:0.2612, validation_loss:0.2987, val_accuracy:0.8831
Epoch [154/200], average_loss:0.2607, validation_loss:0.2973, val_accuracy:0.9221
Epoch [155/200], average_loss:0.2609, validation_loss:0.2993, val_accuracy:0.8831
Epoch [156/200], average_loss:0.2606, validation_loss:0.2993, val_accuracy:0.8961
Epoch [157/200], average_loss:0.2615, validation_loss:0.2998, val_accuracy:0.9091
Epoch [158/200], average_loss:0.2600, validation_loss:0.2995, val_accuracy:0.8312
Epoch [159/200], average_loss:0.2601, validation_loss:0.2974, val_accuracy:0.9481
Epoch [160/200], average_loss:0.2600, validation_loss:0.2980, val_accuracy:0.8961
Epoch [161/200], average_loss:0.2608, validation_loss:0.3006, val_accuracy:0.8831
Epoch [162/200], average_loss:0.2600, validation_loss:0.2977, val_accuracy:0.8831
Epoch [163/200],

In [34]:
m18.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m18.model(torch.tensor(X_test).float().to(m18.device)).cpu().detach(), axis=1)


In [35]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [36]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8439657997106534

In [37]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8971677820267686

In [38]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3551,  540,   24,  120],
       [ 353, 9940,   25,  127],
       [  24,   53,  380,   22],
       [ 243,  178,   12, 1144]], dtype=int64)

In [40]:
m18.tb.close()

In [41]:
#torch.save(m18.model, "models_and_losses/CNN-full_conv.m-II_lr-{1e-3,1e-4}_bs-512.pt")

# test m.IV

In [42]:
CNN_19 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=6, stride=2, padding=0, bias=False), # n*1*160 -> n*16*78
            nn.ReLU(inplace=True),
            layers.CBD1dBlock(in_channels=16, out_channels=32, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*16*78->n*32*38
            layers.CBD1dBlock(in_channels=32, out_channels=64, kernel_size=4, stride=2, bias=False, activation=nn.ReLU(), dropout=0.3),#n*32*38->n*64*18
            layers.Flatten(out_features=64*18),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=64*18, out_features=4)
)

In [43]:
optimizer19= torch.optim.Adam(CNN_19.parameters(), lr=1e-3)

In [44]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [45]:
m19 = Trainer(
        model=CNN_19,
        optimizer=optimizer19,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer19, milestones=[30], gamma=0.1),
        tensorboard=True,
        model_name="CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512",
        verbose=True
        )

cuda:0


In [46]:
lh = m19(epochs=150, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/150], average_loss:0.6858, validation_loss:0.5015, val_accuracy:0.7403
Epoch [2/150], average_loss:0.4783, validation_loss:0.4059, val_accuracy:0.8701
Epoch [3/150], average_loss:0.4219, validation_loss:0.3731, val_accuracy:0.8442
Epoch [4/150], average_loss:0.3981, validation_loss:0.3415, val_accuracy:0.9091
Epoch [5/150], average_loss:0.3745, validation_loss:0.3307, val_accuracy:0.8701
Epoch [6/150], average_loss:0.3731, validation_loss:0.3163, val_accuracy:0.9610
Epoch [7/150], average_loss:0.3592, validation_loss:0.3173, val_accuracy:0.8571
Epoch [8/150], average_loss:0.3524, validation_loss:0.3337, val_accuracy:0.8701
Epoch [9/150], average_loss:0.3508, validation_loss:0.3032, val_accuracy:0.9091
Epoch [10/150], average_loss:0.3438, validation_loss:0.3118, val_accuracy:0.9351
Epoch [11/150], average_loss:0.3386, validation_loss:0.3153, val_accuracy:0.8571
Epoch [12/150], average_loss:0.3324, validation_loss:0.3399, val_accuracy:0.8442
Epoch [13/150], average_loss:0.3297, 

Epoch [103/150], average_loss:0.2554, validation_loss:0.2330, val_accuracy:0.9221
Epoch [104/150], average_loss:0.2547, validation_loss:0.2304, val_accuracy:0.9481
Epoch [105/150], average_loss:0.2546, validation_loss:0.2338, val_accuracy:0.9221
Epoch [106/150], average_loss:0.2552, validation_loss:0.2293, val_accuracy:0.9351
Epoch [107/150], average_loss:0.2558, validation_loss:0.2317, val_accuracy:0.9221
Epoch [108/150], average_loss:0.2549, validation_loss:0.2313, val_accuracy:0.9351
Epoch [109/150], average_loss:0.2562, validation_loss:0.2371, val_accuracy:0.8961
Epoch [110/150], average_loss:0.2558, validation_loss:0.2486, val_accuracy:0.8701
Epoch [111/150], average_loss:0.2556, validation_loss:0.2363, val_accuracy:0.8442
Epoch [112/150], average_loss:0.2563, validation_loss:0.2284, val_accuracy:0.9351
Epoch [113/150], average_loss:0.2569, validation_loss:0.2348, val_accuracy:0.8831
Epoch [114/150], average_loss:0.2550, validation_loss:0.2297, val_accuracy:0.9481
Epoch [115/150],

In [47]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [48]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [49]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8735287699438058

In [50]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9167065009560229

In [51]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3915,  194,   28,   98],
       [ 495, 9782,   35,  133],
       [  17,   30,  417,   15],
       [ 252,   84,   13, 1228]], dtype=int64)

In [53]:
#torch.save(m19.model, "models_and_losses/CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epoch-150.pt")

In [54]:
lh = m19(epochs=range(150,250), train_loader=train_loader, validation_loader=valid_loader)

Epoch [151/250], average_loss:0.2503, validation_loss:0.2380, val_accuracy:0.8831
Epoch [152/250], average_loss:0.2499, validation_loss:0.2282, val_accuracy:0.8701
Epoch [153/250], average_loss:0.2520, validation_loss:0.2273, val_accuracy:0.8961
Epoch [154/250], average_loss:0.2497, validation_loss:0.2222, val_accuracy:1.0000
Epoch [155/250], average_loss:0.2526, validation_loss:0.2251, val_accuracy:0.9481
Epoch [156/250], average_loss:0.2502, validation_loss:0.2299, val_accuracy:0.9481
Epoch [157/250], average_loss:0.2502, validation_loss:0.2248, val_accuracy:0.9351
Epoch [158/250], average_loss:0.2506, validation_loss:0.2221, val_accuracy:0.9610
Epoch [159/250], average_loss:0.2498, validation_loss:0.2271, val_accuracy:0.8961
Epoch [160/250], average_loss:0.2506, validation_loss:0.2317, val_accuracy:0.8701
Epoch [161/250], average_loss:0.2482, validation_loss:0.2259, val_accuracy:0.9351
Epoch [162/250], average_loss:0.2493, validation_loss:0.2275, val_accuracy:0.9221
Epoch [163/250],

In [55]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [56]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [57]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8766860982688688

In [58]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9202318355640535

In [59]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3889,  207,   29,  110],
       [ 425, 9841,   36,  143],
       [  15,   30,  418,   16],
       [ 228,   84,   12, 1253]], dtype=int64)

In [60]:
#torch.save(m19.model, "models_and_losses/CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epoch-250.pt")

In [61]:
lh = m19(epochs=range(250,350), train_loader=train_loader, validation_loader=valid_loader)

Epoch [251/350], average_loss:0.2410, validation_loss:0.2166, val_accuracy:0.9351
Epoch [252/350], average_loss:0.2420, validation_loss:0.2193, val_accuracy:0.8701
Epoch [253/350], average_loss:0.2428, validation_loss:0.2299, val_accuracy:0.8571
Epoch [254/350], average_loss:0.2421, validation_loss:0.2200, val_accuracy:0.9351
Epoch [255/350], average_loss:0.2409, validation_loss:0.2165, val_accuracy:0.9351
Epoch [256/350], average_loss:0.2414, validation_loss:0.2229, val_accuracy:0.8961
Epoch [257/350], average_loss:0.2409, validation_loss:0.2178, val_accuracy:0.9610
Epoch [258/350], average_loss:0.2437, validation_loss:0.2174, val_accuracy:0.9221
Epoch [259/350], average_loss:0.2409, validation_loss:0.2213, val_accuracy:0.8442
Epoch [260/350], average_loss:0.2410, validation_loss:0.2140, val_accuracy:0.9740
Epoch [261/350], average_loss:0.2398, validation_loss:0.2206, val_accuracy:0.9091
Epoch [262/350], average_loss:0.2404, validation_loss:0.2302, val_accuracy:0.9221
Epoch [263/350],

In [62]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [63]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [64]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8777875503329408

In [65]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9248326959847036

In [66]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3878,  237,   30,   90],
       [ 321, 9947,   41,  136],
       [  19,   27,  418,   15],
       [ 248,   80,   14, 1235]], dtype=int64)

In [67]:
#torch.save(m19.model, "models_and_losses/CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epoch-350.pt")

## pokus

In [68]:
for param_group in m19.optimizer.param_groups:
        param_group['lr'] = 1e-3

In [69]:
lh = m19(epochs=range(350,380), train_loader=train_loader, validation_loader=valid_loader)

Epoch [351/380], average_loss:0.2589, validation_loss:0.2328, val_accuracy:0.8831
Epoch [352/380], average_loss:0.2381, validation_loss:0.2182, val_accuracy:0.9091
Epoch [353/380], average_loss:0.2362, validation_loss:0.2203, val_accuracy:0.9221
Epoch [354/380], average_loss:0.2378, validation_loss:0.2156, val_accuracy:0.9481
Epoch [355/380], average_loss:0.2380, validation_loss:0.2136, val_accuracy:0.9221
Epoch [356/380], average_loss:0.2344, validation_loss:0.2148, val_accuracy:0.9221
Epoch [357/380], average_loss:0.2362, validation_loss:0.2128, val_accuracy:0.9351
Epoch [358/380], average_loss:0.2379, validation_loss:0.2149, val_accuracy:0.9091
Epoch [359/380], average_loss:0.2331, validation_loss:0.2127, val_accuracy:0.9610
Epoch [360/380], average_loss:0.2377, validation_loss:0.2169, val_accuracy:0.9351
Epoch [361/380], average_loss:0.2343, validation_loss:0.2121, val_accuracy:0.9351
Epoch [362/380], average_loss:0.2361, validation_loss:0.2209, val_accuracy:0.9221
Epoch [363/380],

In [70]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [71]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [72]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.877756322430652

In [73]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9196343212237094

In [74]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3924,  159,   31,  121],
       [ 494, 9771,   35,  145],
       [  17,   23,  424,   15],
       [ 225,   64,   16, 1272]], dtype=int64)

In [75]:
for param_group in m19.optimizer.param_groups:
        param_group['lr'] = 1e-4

In [76]:
lh = m19(epochs=range(380,420), train_loader=train_loader, validation_loader=valid_loader)

Epoch [381/420], average_loss:0.2333, validation_loss:0.2125, val_accuracy:0.9351
Epoch [382/420], average_loss:0.2334, validation_loss:0.2096, val_accuracy:0.9221
Epoch [383/420], average_loss:0.2329, validation_loss:0.2158, val_accuracy:0.8571
Epoch [384/420], average_loss:0.2299, validation_loss:0.2142, val_accuracy:0.8701
Epoch [385/420], average_loss:0.2346, validation_loss:0.2153, val_accuracy:0.9221
Epoch [386/420], average_loss:0.2348, validation_loss:0.2150, val_accuracy:0.8961
Epoch [387/420], average_loss:0.2316, validation_loss:0.2136, val_accuracy:0.8961
Epoch [388/420], average_loss:0.2320, validation_loss:0.2136, val_accuracy:0.8571
Epoch [389/420], average_loss:0.2334, validation_loss:0.2113, val_accuracy:0.9221
Epoch [390/420], average_loss:0.2319, validation_loss:0.2063, val_accuracy:0.9740
Epoch [391/420], average_loss:0.2311, validation_loss:0.2181, val_accuracy:0.9091
Epoch [392/420], average_loss:0.2334, validation_loss:0.2119, val_accuracy:0.9091
Epoch [393/420],

In [77]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [78]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [79]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.8769967959807636

In [80]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.924593690248566

In [81]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[ 3847,   276,    30,    82],
       [  289, 10012,    32,   112],
       [   18,    34,   414,    13],
       [  265,    97,    14,  1201]], dtype=int64)

In [82]:
#torch.save(m19.model, "models_and_losses/CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epoch-420.pt")

In [83]:
lh = m19(epochs=range(420,450), train_loader=train_loader, validation_loader=valid_loader)

Epoch [421/450], average_loss:0.2315, validation_loss:0.2072, val_accuracy:0.9221
Epoch [422/450], average_loss:0.2293, validation_loss:0.2069, val_accuracy:0.9610
Epoch [423/450], average_loss:0.2317, validation_loss:0.2117, val_accuracy:0.8961
Epoch [424/450], average_loss:0.2325, validation_loss:0.2069, val_accuracy:0.9351
Epoch [425/450], average_loss:0.2308, validation_loss:0.2100, val_accuracy:0.8961
Epoch [426/450], average_loss:0.2309, validation_loss:0.2134, val_accuracy:0.9091
Epoch [427/450], average_loss:0.2324, validation_loss:0.2193, val_accuracy:0.9221
Epoch [428/450], average_loss:0.2317, validation_loss:0.2052, val_accuracy:0.9740
Epoch [429/450], average_loss:0.2310, validation_loss:0.2125, val_accuracy:0.9351
Epoch [430/450], average_loss:0.2293, validation_loss:0.2126, val_accuracy:0.9740
Epoch [431/450], average_loss:0.2310, validation_loss:0.2135, val_accuracy:0.8961
Epoch [432/450], average_loss:0.2305, validation_loss:0.2177, val_accuracy:0.8831
Epoch [433/450],

In [84]:
m19.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m19.model(torch.tensor(X_test).float().to(m19.device)).cpu().detach(), axis=1)


In [85]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [86]:
f1_score(y_true=y_test, y_pred=x_pred, average="macro")

0.880439326282784

In [87]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9228011472275335

In [88]:
confusion_matrix(y_true=y_test, y_pred=x_pred)

array([[3920,  172,   29,  114],
       [ 445, 9847,   31,  122],
       [  18,   27,  420,   14],
       [ 221,   84,   15, 1257]], dtype=int64)

In [89]:
#torch.save(m19.model, "models_and_losses/CNN-m.IV-Conv1-BN-Drop-0.3_ReLU_lr-{1e-3,1e-4}_bs-512_epoch-450.pt")